In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

In [2]:
class CurrencyConverterInput(BaseModel):
    """Input schema for CurrencyConverterTool."""
    amount: float = Field(..., description="The amount to convert.")
    from_currency: str = Field(..., description="The source currency code (e.g., 'USD').")
    to_currency: str = Field(..., description="The target currency code (e.g., 'EUR').")


In [5]:
class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another."
    args_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = os.getenv("EXCHANGE_RATE_API_KEY") 

    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        response = requests.get(url)
        
        if response.status_code != 200:
            return "Failed to fetch exchange rates."

        data = response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"

        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equivalent to {converted_amount:.2f} {to_currency}."


In [6]:
from crewai import Agent

currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversions and financial insights.",
    backstory=(
        "You are a finance expert with deep knowledge of global exchange rates."
        "You help users with currency conversion and financial decision-making."
    ),
    tools=[CurrencyConverterTool()],  # Attach our custom tool
    verbose=True
)

In [7]:
from crewai import Task

currency_conversion_task = Task(
    description=(
        "Convert {amount} {from_currency} to {to_currency} "
        "using real-time exchange rates."
        "Provide the equivalent amount and "
        "explain any relevant financial context."
    ),
    expected_output=("A detailed response including the "
                     "converted amount and financial insights."),
    agent=currency_analyst
)

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(inputs={"amount": 100, 
                                "from_currency": "USD",
                                "to_currency": "INR"})


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Currency Analyst                                                                                        │
│                                                                                                                 │
│  Task: Convert 100 USD to INR using real-time exchange rates.Provide the equivalent amount and explain any      │
│  relevant financial context.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Currency Analyst                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to convert 100 USD to INR using real-time exchange rates to provide the equivalent    │
│  amount. Then I will provide relevant financial context regarding the conversion.                               │
│                                                                                                                 │
│  Using Tool: Currency Converter Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "amount": 100,                                                                                               │
│    "from_currency": "USD",                                                                                      │
│    "to_currency": "INR"                                                                                         │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  100.0 USD is equivalent to 8945.39 INR.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Currency Analyst                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Converting 100 USD to INR at the current real-time exchange rate results in approximately 8945.39 INR. The     │
│  USD to INR exchange rate is influenced by factors such as interest rate differentials, inflation               │
│  expectations, trade balance, and geopolitical developments. The Indian Rupee can be sensitive to changes in    │
│  the US dollar value due to India's trade relations with the US, foreign investment flows, and currency demand  │
│  in global markets. Given the current rate, 100 USD holds significant value in INR, reflecting economic         │
│  conditions and currency market dynamics between the two countries.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
Would you like to view your execution traces? [y/N] (20s timeout): 



┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                                             │
│  Info: Tracing has been disabled.                                           │
│                                                                             │
│  Your preference has been saved. Future Crew/Flow executions will not       │
│  collect traces.                                                            │
│                                                                             │
│  To enable tracing later, do any one of these:                              │
│  • Set tracing=True in your Crew/Flow code                                  │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file              │
│  • Run: crewai traces enable                                                │
│                                                                             │
└─────────────────────────────────────

In [10]:
from IPython.display import Markdown
Markdown(response.raw)

Converting 100 USD to INR at the current real-time exchange rate results in approximately 8945.39 INR. The USD to INR exchange rate is influenced by factors such as interest rate differentials, inflation expectations, trade balance, and geopolitical developments. The Indian Rupee can be sensitive to changes in the US dollar value due to India's trade relations with the US, foreign investment flows, and currency demand in global markets. Given the current rate, 100 USD holds significant value in INR, reflecting economic conditions and currency market dynamics between the two countries.